In [1]:
!pip install TPOT

In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import VotingClassifier

from sklearn.semi_supervised import SelfTrainingClassifier

from sklearn.metrics import classification_report

from tqdm import tqdm


In [3]:
from tpot import TPOTClassifier

/opt/conda/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [4]:
!pip install xgboost

In [5]:
from xgboost import XGBClassifier

In [6]:
from nlp_preprocessing import TrainTransfromVect

## 데이터 로드

In [7]:
train_df=pd.read_csv('ratings_train.csv')

In [8]:
test_df = pd.read_csv('ratings_test.csv')

In [9]:
train_df.dropna(inplace=True)

In [10]:
test_df.dropna(inplace=True)

In [11]:
train_df

,Unnamed: 0,id,document,label
0,0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,2,10265843,너무재밓었다그래서보는것을추천한다,0
3,3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...,...
149995,149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,149996,8549745,평점이 너무 낮아서...,1
149997,149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [12]:
train_df['label'].unique(), len(train_df)

(array([0, 1]), 149995)

In [13]:
test_df['label'].unique(), len(test_df)

(array([1, 0]), 49997)

### 분석 모델 훈련에서 사용할 옵션값

In [14]:
## 모델 훈련시 사용할 랜덤 변수(계속 모델 훈련해도 고정된 결과가 나옴)
random_seed_num = 42


In [15]:
train_df_num=10000 #훈련 데이터 중에서 훈련에 사용할 데이터 수

In [16]:
test_df_num= 5000 #테스트 데이터 중에서 훈련에 사용할 데이터 수

## 데이터 전처리

1. 데이터 분리
    1) X, y 데이터 분리 (train과 test 데이터 분리: 네이버 영화 리뷰는 이미 분리되어 해당프로세스 생략)
    2) X 데이터 벡터화

In [17]:
# 1. 데이터 분리
x_train, y_train = train_df['document'][:train_df_num], train_df['label'][:train_df_num]
x_test, y_test = test_df['document'][:test_df_num], test_df['label'][:test_df_num]

In [18]:
## 인덱스로 데이터를 분리하여, 훈련데이터와 테스트 데이터의 0과 1의 수를 확인
print(train_df[:train_df_num].groupby('label').count())
print(test_df[:test_df_num].groupby('label').count())

       Unnamed: 0    id  document
label                            
0            5021  5021      5021
1            4979  4979      4979
       Unnamed: 0    id  document
label                            
0            2478  2478      2478
1            2522  2522      2522


#### 분류모델에서 y값의 순서, 분포에 따라 모델 학습 결과가 달라질수 있다.
ex) y=[0,0,0,0,0,0,0,0,0,....0,0,1,1,1,1,1,1,1,1,]

    모델 학습시 y1=[0,0,0,0,0,]. y2=[0,0,0,0,0,] y3= ...., yn-1=[1,1,1,1,1,] yn=[1,1,1,1,]
    1) 1번 모델은 0을 예측, 2모델 0을 예측... n-1인 경우에는 1만 예측...
    2) 각 모델들은 극단적인 값만 학습해서 최종적으로 앙상블 된 모델 결과가 않 좋을 수 있음

#### 분류 모델의 데이터를 넣는 방법
ex) y= [0,1,0,1,0,1,0,1,....]
    y= [0,1,2,0,1,2..2,2,2,1,0,1,1,,0,2]

* 간편하게 하는 방법
    * sklearn.model_selection.train_test_split(~ , shuffle=True)'
    
* 특정 아이디에 따른 예측이 필요한경우
    * 모델 훈련 < "서비스 모델 적용"
        * sklearn.model_selection.train_test_split 파리미터 'shuffle=False'
        * 이미 저장된 모델을 이용해서 predict를 하는 경우는 상관이 없음
        

In [19]:
## 커스텀화한 벡터 훈련 + 변경 클래스 선언

In [20]:
tv = TrainTransfromVect()

In [21]:
tv.fit_run('wp',x_train)

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [22]:
wp_vec_train = tv.transform_run(x_train, chunk_size=100)
wp_vec_test = tv.transform_run(x_test, chunk_size=100)

100%|███████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 35.53it/s]


In [23]:
tv.fit_run('bpe',x_train)
bpe_vec_train = tv.transform_run(x_train, chunk_size=100)
bpe_vec_test = tv.transform_run(x_test, chunk_size=100)

100%|███████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 33.52it/s]


In [24]:
#tv.fit_run('kkma',x_train)
#kkma_vec_train = tv.transform_run(x_train, chunk_size=100)
#kkma_vec_test = tv.transform_run(x_test, chunk_size=100)

In [25]:
#tv.fit_run('mp',x_train)
#mp_vec_train = tv.transform_run(x_train, chunk_size=100)
#mp_vec_test = tv.transform_run(x_test, chunk_size=100)

### 긍정과 부정을 예측하는 분류모델을 학습

In [26]:
clf_decision= DecisionTreeClassifier(random_state=random_seed_num, max_depth=7)

In [27]:
clf_decision.fit(wp_vec_train,y_train)

DecisionTreeClassifier(max_depth=7, random_state=42)

In [28]:
#워드 피스 토크나이저 적용
y_train_pred= clf_decision.predict(wp_vec_train)
y_test_pred= clf_decision.predict(wp_vec_test)

In [29]:
print(classification_report(y_train,y_train_pred))

              precision    recall  f1-score   support

           0       0.55      0.96      0.70      5021
           1       0.85      0.20      0.33      4979

    accuracy                           0.59     10000
   macro avg       0.70      0.58      0.51     10000
weighted avg       0.70      0.59      0.52     10000



In [30]:
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.54      0.95      0.69      2478
           1       0.80      0.19      0.30      2522

    accuracy                           0.57      5000
   macro avg       0.67      0.57      0.49      5000
weighted avg       0.67      0.57      0.49      5000



#### BPE 토크나이저로 벡터화한 x 데이터 모델 훈련

In [31]:
clf_decision= DecisionTreeClassifier(random_state=random_seed_num, max_depth=7)

In [32]:
clf_decision.fit(bpe_vec_train,y_train)
y_train_pred= clf_decision.predict(bpe_vec_train)
y_test_pred= clf_decision.predict(bpe_vec_test)

In [33]:
print(classification_report(y_train,y_train_pred))

              precision    recall  f1-score   support

           0       0.55      0.96      0.70      5021
           1       0.84      0.19      0.31      4979

    accuracy                           0.58     10000
   macro avg       0.69      0.58      0.50     10000
weighted avg       0.69      0.58      0.50     10000



In [34]:
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.53      0.95      0.68      2478
           1       0.78      0.17      0.28      2522

    accuracy                           0.56      5000
   macro avg       0.66      0.56      0.48      5000
weighted avg       0.66      0.56      0.48      5000



## 랜덤 포레스트 분류 모델

In [35]:
rfc_decision= RandomForestClassifier(random_state=random_seed_num, max_depth=7)
rfc_decision.fit(wp_vec_train,y_train)
y_train_pred= rfc_decision.predict(wp_vec_train)
y_test_pred= rfc_decision.predict(wp_vec_test)
print(classification_report(y_train,y_train_pred))
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.70      0.88      0.78      5021
           1       0.84      0.63      0.72      4979

    accuracy                           0.75     10000
   macro avg       0.77      0.75      0.75     10000
weighted avg       0.77      0.75      0.75     10000

              precision    recall  f1-score   support

           0       0.66      0.83      0.74      2478
           1       0.78      0.58      0.66      2522

    accuracy                           0.70      5000
   macro avg       0.72      0.71      0.70      5000
weighted avg       0.72      0.70      0.70      5000



#### Kneighbors 분류모델

* 언발란스 데이터에 대한 분류모델로 성능이 좋은 편임
 * 비대칭 데이터 관련 패키지 참조(10프로 미만)
 * 현재 데이터에서는 과적합되어잇는 것으로 보임

In [36]:
clf_kneighbors= KNeighborsClassifier(n_neighbors=2, weights='distance', leaf_size=50)
clf_kneighbors.fit(wp_vec_train,y_train)
y_train_pred= clf_kneighbors.predict(wp_vec_train)
y_test_pred= clf_kneighbors.predict(wp_vec_test)

### 트레이닝 에서 분석결과가 너무 좋으면 의심해야 한다.
print(classification_report(y_train,y_train_pred))
## 테스트에서 떨어짐. 훈련데이터에 너무 학습된 모델임
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5021
           1       1.00      0.99      1.00      4979

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000

              precision    recall  f1-score   support

           0       0.68      0.68      0.68      2478
           1       0.69      0.68      0.69      2522

    accuracy                           0.68      5000
   macro avg       0.68      0.68      0.68      5000
weighted avg       0.68      0.68      0.68      5000



### LogisticRegression 분류모델

In [39]:
clf_logistic= LogisticRegression(max_iter=5000,random_state=random_seed_num)
clf_logistic.fit(wp_vec_train,y_train)
y_train_pred= clf_logistic.predict(wp_vec_train)
y_test_pred= clf_logistic.predict(wp_vec_test)
print(classification_report(y_train,y_train_pred))
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91      5021
           1       0.92      0.90      0.91      4979

    accuracy                           0.91     10000
   macro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000

              precision    recall  f1-score   support

           0       0.79      0.82      0.80      2478
           1       0.81      0.78      0.80      2522

    accuracy                           0.80      5000
   macro avg       0.80      0.80      0.80      5000
weighted avg       0.80      0.80      0.80      5000



In [40]:
clf_logistic= LogisticRegression(max_iter=5000,random_state=random_seed_num)
clf_logistic.fit(bpe_vec_train,y_train)
y_train_pred= clf_logistic.predict(bpe_vec_train)
y_test_pred= clf_logistic.predict(bpe_vec_test)
print(classification_report(y_train,y_train_pred))
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91      5021
           1       0.92      0.90      0.91      4979

    accuracy                           0.91     10000
   macro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000

              precision    recall  f1-score   support

           0       0.78      0.82      0.80      2478
           1       0.81      0.78      0.79      2522

    accuracy                           0.80      5000
   macro avg       0.80      0.80      0.80      5000
weighted avg       0.80      0.80      0.80      5000



## MLP

In [41]:
clf_mlp= MLPClassifier(solver='lbfgs',alpha=0.5,max_iter=100,random_state=random_seed_num)
clf_mlp.fit(wp_vec_train,y_train)
y_train_pred= clf_mlp.predict(wp_vec_train)
y_test_pred= clf_mlp.predict(wp_vec_test)
print(classification_report(y_train,y_train_pred))
print(classification_report(y_test,y_test_pred))

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5021
           1       0.99      1.00      1.00      4979

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000

              precision    recall  f1-score   support

           0       0.78      0.79      0.78      2478
           1       0.79      0.78      0.78      2522

    accuracy                           0.78      5000
   macro avg       0.78      0.78      0.78      5000
weighted avg       0.78      0.78      0.78      5000



### Xgboost

In [42]:
clf_xgb= XGBClassifier(random_state=random_seed_num, max_depth=7)
clf_xgb.fit(wp_vec_train,y_train)
y_train_pred= clf_xgb.predict(wp_vec_train)
y_test_pred= clf_xgb.predict(wp_vec_test)
print(classification_report(y_train,y_train_pred))
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.81      0.90      0.85      5021
           1       0.89      0.78      0.83      4979

    accuracy                           0.84     10000
   macro avg       0.85      0.84      0.84     10000
weighted avg       0.85      0.84      0.84     10000

              precision    recall  f1-score   support

           0       0.69      0.80      0.74      2478
           1       0.77      0.65      0.70      2522

    accuracy                           0.72      5000
   macro avg       0.73      0.72      0.72      5000
weighted avg       0.73      0.72      0.72      5000



### 어떤 모델,  어떤 백터가 좋은 예측 모델을 만들었는지 확인

1. wp,bpe, kkma, mp(okt)
    * wp: 분류모델에 사용하기 적합( 다른 토크나이저보다 소요시간, 모델성능에서 유리함)
    * bpe : kkma, mp 보다는 모델 훈련에서 시간소요가 덜하지만, wp 보다는 모델 성능에서 효과가 낮음 (의사결정, 로지스틱)
    * kkma :모델 훈련 적용 불가
    * mp: 모델 훈련 적용 불가
    
2. 분류 모델 중에서 사용할만한 모델 선택
    * DecisionTreeClassifier(의사결정나무): 모델의 성능평가가 가장 낮음
    * RandomForestClassifier : 의사결정보다는 높은 모델 성능 평가, 하지만 다른 모델보다는 성능이 낮음
    * xgboost : 모델 성능에서는 활용할 수 있을 정도로 판단됨. 다른 모델보다 훈련 시간이 필요함 (과적합 낮은 편)
    * LogisticRegression : 다른 모델보다 적합한 모델로 보임 (과적합 낮은편, 모델 성능 준수한 편)
    * MLPClassifier : 모델의 과적합 된것으로 보임 (f1-score : train=test+0.2)
    * KNeighborsClassifier : 모델이 과적합된것으로 보임 (f1-score : train=test+0.3)



### 앙상블 분류 모델

- 앙상블 1: 랜덤포레스트 / xgboost/ 로지스틱

- 앙상블 2: mlp/xgboost/로지스틱

In [44]:
clf_randomf = RandomForestClassifier(random_state=random_seed_num, max_depth=7)
clf_xgboost= XGBClassifier(random_state=random_seed_num, max_depth=7)
clf_logistic= LogisticRegression(max_iter=5000,random_state=random_seed_num)
clf_ensemble_1=VotingClassifier(estimators=[('xgoboost',clf_xgboost),('randomforest',clf_randomf),('logistic',clf_logistic)], voting='hard')

In [45]:
clf_ensemble_1.fit(wp_vec_train,y_train)
y_train_pred= clf_ensemble_1.predict(wp_vec_train)
y_test_pred= clf_ensemble_1.predict(wp_vec_test)
print(classification_report(y_train,y_train_pred))
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.81      0.91      0.86      5021
           1       0.90      0.78      0.84      4979

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000

              precision    recall  f1-score   support

           0       0.71      0.84      0.77      2478
           1       0.81      0.66      0.73      2522

    accuracy                           0.75      5000
   macro avg       0.76      0.75      0.75      5000
weighted avg       0.76      0.75      0.75      5000



In [46]:
clf_mlp= MLPClassifier(solver='lbfgs',alpha=0.5,max_iter=100,random_state=random_seed_num)
clf_xgboost= XGBClassifier(random_state=random_seed_num, max_depth=7)
clf_logistic= LogisticRegression(max_iter=5000,random_state=random_seed_num)
clf_ensemble_2=VotingClassifier(estimators=[('xgoboost',clf_xgboost),('mlp',clf_mlp),('logistic',clf_logistic)], voting='hard')

In [47]:
clf_ensemble_2.fit(wp_vec_train,y_train)
y_train_pred= clf_ensemble_2.predict(wp_vec_train)
y_test_pred= clf_ensemble_2.predict(wp_vec_test)
print(classification_report(y_train,y_train_pred))
print(classification_report(y_test,y_test_pred))

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


              precision    recall  f1-score   support

           0       0.93      0.96      0.94      5021
           1       0.95      0.93      0.94      4979

    accuracy                           0.94     10000
   macro avg       0.94      0.94      0.94     10000
weighted avg       0.94      0.94      0.94     10000

              precision    recall  f1-score   support

           0       0.78      0.82      0.80      2478
           1       0.81      0.77      0.79      2522

    accuracy                           0.80      5000
   macro avg       0.80      0.80      0.80      5000
weighted avg       0.80      0.80      0.80      5000



#### 모델 훈련 및 모델을 선택하기

1. 개별 모델을 하나씩 훈련시키고 평가
2. 각 모델들의 성능을 비교해서 앙상블 모델로 만들기
    - 알고리즘끼리 유사한 모델들은 제외
    - 모델 훈련 시간이 오래걸리는 것은 제외
    - hardvoting : 각 모델의 분류된 라벨에 대한 voting, 다수결
    - softvoting : 각 모델에서 라벨에 대한 예측 확률

### 준지도학습

* 지도학습 base 모델로 사용
* y 라벨 일부 masking -> masking label을 예측 -> 훈련 데이터셋 활용
* 이유: 새로운 데이터에 예측율을 높히기 위해

In [ ]:
from sklearn.semi_supervised  import SelfTrainingClassifier

In [48]:
semi_clf_xgboost= XGBClassifier(random_state=random_seed_num, max_depth=7)
self_train_model= SelfTrainingClassifier(semi_clf_xgboost)

In [49]:
self_train_model.fit(wp_vec_train,y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/semi_supervised/_self_training.py:212: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


SelfTrainingClassifier(base_estimator=XGBClassifier(base_score=None,
                                                    booster=None,
                                                    callbacks=None,
                                                    colsample_bylevel=None,
                                                    colsample_bynode=None,
                                                    colsample_bytree=None,
                                                    early_stopping_rounds=None,
                                                    enable_categorical=False,
                                                    eval_metric=None,
                                                    feature_types=None,
                                                    gamma=None, gpu_id=None,
                                                    grow_policy=None,
                                                    importance_type=None,
                                                    interaction_constraints=None,
                                                    learning_rate=None,
                                                    max_bin=None,
                                                    max_cat_threshold=None,
                                                    max_cat_to_onehot=None,
                                                    max_delta_step=None,
                                                    max_depth=7,
                                                    max_leaves=None,
                                                    min_child_weight=None,
                                                    missing=nan,
                                                    monotone_constraints=None,
                                                    n_estimators=100,
                                                    n_jobs=None,
                                                    num_parallel_tree=None,
                                                    predictor=None,
                                                    random_state=42, ...))

In [50]:
y_train_pred=self_train_model.predict(wp_vec_train)
y_test_pred=self_train_model.predict(wp_vec_test)

In [51]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.81      0.90      0.85      5021
           1       0.89      0.78      0.83      4979

    accuracy                           0.84     10000
   macro avg       0.85      0.84      0.84     10000
weighted avg       0.85      0.84      0.84     10000



In [52]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.69      0.80      0.74      2478
           1       0.77      0.65      0.70      2522

    accuracy                           0.72      5000
   macro avg       0.73      0.72      0.72      5000
weighted avg       0.73      0.72      0.72      5000



In [53]:
semi_clf_dc= DecisionTreeClassifier(random_state=random_seed_num, max_depth=7)
self_train_model2= SelfTrainingClassifier(semi_clf_dc)

In [55]:
self_train_model2.fit(wp_vec_train,y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/semi_supervised/_self_training.py:212: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


SelfTrainingClassifier(base_estimator=DecisionTreeClassifier(max_depth=7,
                                                             random_state=42))

In [56]:
y_train_pred=self_train_model2.predict(wp_vec_train)
y_test_pred=self_train_model2.predict(wp_vec_test)

In [57]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.55      0.96      0.70      5021
           1       0.85      0.20      0.33      4979

    accuracy                           0.59     10000
   macro avg       0.70      0.58      0.51     10000
weighted avg       0.70      0.59      0.52     10000



In [58]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.54      0.95      0.69      2478
           1       0.80      0.19      0.30      2522

    accuracy                           0.57      5000
   macro avg       0.67      0.57      0.49      5000
weighted avg       0.67      0.57      0.49      5000



### 준지도 학습모델을 사용했던 경험

1. 지도학습과 준지도학습 모델 훈련 결과 비슷
    -> 데이터 양이 많음
    -> 전처리에서 이미 주요 피쳐 추출에서 모델 훈련
    
2. 지도학습과 준지도학습의 모델 훈련 결과가 차이가 난 경우
    -> 데이터양이 적음
    -> 준지도학습에서 성능이 떨어진경우, 비대칭 데이터의 경우 다수 발생
    -> 약간의 모델 성능, test에서 좀더 좋은 성능 내는 편임

#### TPOT AutoML

In [38]:
# tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, random_state=42)
# tpot.fit(wp_vec_train, y_train)
# print(tpot.score(wp_vec_test, y_test))